# Face swapping

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir Face_swapping
%cd Face_swapping
import gdown
!gdown --id 1y6IUt3WOF5LUjnLqlBNxjJrx7OObc5gq

/content/Face_swapping
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1y6IUt3WOF5LUjnLqlBNxjJrx7OObc5gq
To: /content/Face_swapping/deepfake.zip
100% 1.69G/1.69G [00:20<00:00, 80.6MB/s]


In [6]:
!unzip /content/Face_swapping/deepfake.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Validation/Real/real_5499.jpg  
  inflating: Dataset/Validation/Real/real_55.jpg  
  inflating: Dataset/Validation/Real/real_550.jpg  
  inflating: Dataset/Validation/Real/real_5500.jpg  
  inflating: Dataset/Validation/Real/real_5501.jpg  
  inflating: Dataset/Validation/Real/real_5502.jpg  
  inflating: Dataset/Validation/Real/real_5503.jpg  
  inflating: Dataset/Validation/Real/real_5504.jpg  
  inflating: Dataset/Validation/Real/real_5505.jpg  
  inflating: Dataset/Validation/Real/real_5506.jpg  
  inflating: Dataset/Validation/Real/real_5507.jpg  
  inflating: Dataset/Validation/Real/real_5508.jpg  
  inflating: Dataset/Validation/Real/real_5509.jpg  
  inflating: Dataset/Validation/Real/real_551.jpg  
  inflating: Dataset/Validation/Real/real_5510.jpg  
  inflating: Dataset/Validation/Real/real_5511.jpg  
  inflating: Dataset/Validation/Real/real_5512.jpg  
  inflating: Dataset/Validation/Real/real_5513.jpg  

In [ ]:
!nvidia-smi

!nvcc --version

Sat Jun 24 06:56:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
 #**Clone github & download models**

!git clone https://github.com/sberbank-ai/sber-swap.git
%cd sber-swap

# load arcface
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/iresnet.py

# load landmarks detector
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/glintr100.onnx
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/scrfd_10g_bnkps.onnx

# load model itself
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/sber-swap-v2.0/G_unet_2blocks.pth

# load super res model
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/super-res/10_net_G.pth

Cloning into 'sber-swap'...
remote: Enumerating objects: 704, done.
remote: Total 704 (delta 0), reused 0 (delta 0), pack-reused 704
Receiving objects: 100% (704/704), 94.00 MiB | 38.04 MiB/s, done.
Resolving deltas: 100% (176/176), done.
/content/Face_swapping/sber-swap
--2023-06-26 06:59:25--  https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ai-forever/ghost/releases/download/arcface/backbone.pth [following]
--2023-06-26 06:59:25--  https://github.com/ai-forever/ghost/releases/download/arcface/backbone.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/437939371/bfc1b827-8d3a-4001-863c-d6d04ffb1258?X-Amz-Algorit

In [8]:
#@markdown #**Install required libraries**

!pip install mxnet-cu112
!pip install onnxruntime-gpu==1.12
!pip install insightface==0.2.1
!pip install kornia==0.5.4
!pip install dill

!rm /usr/local/lib/python3.10/dist-packages/insightface/model_zoo/model_zoo.py #change the path to python in case you use a different version
!wget -P /usr/local/lib/python3.10/dist-packages/insightface/model_zoo/ https://github.com/AlexanderGroshev/insightface/releases/download/model_zoo/model_zoo.py #change the path to python in case you use a different version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 MB 709.3 kB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 65.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
#@markdown #**Preparation**

import cv2
import torch
import time
import os

from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


OSError: ignored

In [ ]:
#@markdown #**Initialize models**

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

# one vs one image swapping

In [ ]:
#@markdown #**Upload source image**

source_path = 'examples/images/elon_musk.jpg' #@param {type:"string"}

source_full = cv2.imread(source_path)
crop_size = 224  # don't change this

# Check if we can detect a face on the source image
try:
    source = crop_face(source_full, app, crop_size)[0]
    source = [source[:, :, ::-1]]
    print("Everything is ok!")
except TypeError:
    print("Bad source image")

# Specify the path to the target image
target_path = 'examples/images/beckham.jpg' #@param {type:"string"}
target_full = cv2.imread(target_path)
target = crop_face(target_full, app, crop_size)[0]
target = [target[:, :, ::-1]]

# Create a list of full_frames with the target image
full_frames = [target_full]

# Inference
batch_size = 40 #@param {type:"integer"}
START_TIME = time.time()

final_frames_list, crop_frames_list, _, tfm_array_list = model_inference(
    full_frames,
    source,
    target,
    netArc,
    G,
    app,
    set_target=False,
    crop_size=crop_size,
    BS=batch_size
)

if use_sr:
    final_frames_list = face_enhancement(final_frames_list, model)

result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)
cv2.imwrite('examples/results/result.png', result)


# Folder vs Folder swapping

In [ ]:
%cd /content/Face_swapping
!mkdir swapping_results
%cd /content/Face_swapping/sber-swap

/content/Face_swapping
/content/Face_swapping/sber-swap


In [ ]:
 #**Upload source images and target images**

source_folder = '/content/Face_swapping/Deepfake_dataset/source'  # Path to the folder containing source images
target_folder = '/content/Face_swapping/Deepfake_dataset/target'  # Path to the folder containing target images

output_folder = '/content/Face_swapping/swapping_results'  # Path to the folder to save the resulting images

crop_size = 224  # Don't change this

# Get the list of source and target images
source_images = sorted(os.listdir(source_folder))
target_images = sorted(os.listdir(target_folder))

# Check if the number of source and target images is the same
if len(source_images) != len(target_images):
    print("Number of source images does not match the number of target images.")
    exit()

# Iterate over the images
for i in range(len(source_images)):
    source_image_file = source_images[i]
    target_image_file = target_images[i]

    source_path = os.path.join(source_folder, source_image_file)
    target_path = os.path.join(target_folder, target_image_file)

    source_full = cv2.imread(source_path)
    target_full = cv2.imread(target_path)

    # Check if we can detect a face on the source image
    try:
        source = crop_face(source_full, app, crop_size)[0]
        source = [source[:, :, ::-1]]
        print(f"Processing source image: {source_image_file}")
    except TypeError:
        print(f"Skipping source image: {source_image_file}")
        continue

    # Check if we can detect a face on the target image
    try:
        target = crop_face(target_full, app, crop_size)[0]
        target = [target[:, :, ::-1]]
        print(f"Processing target image: {target_image_file}")
    except TypeError:
        print(f"Skipping target image: {target_image_file}")
        continue

    # Inference
    batch_size = 40  # Adjust this if needed
    START_TIME = time.time()

    full_frames = [target_full]  # Define full_frames here based on target_full

    final_frames_list, crop_frames_list, _, tfm_array_list = model_inference(
        full_frames,
        source,
        target,
        netArc,
        G,
        app,
        set_target=False,
        crop_size=crop_size,
        BS=batch_size
    )

    if use_sr:
        final_frames_list = face_enhancement(final_frames_list, model)

    result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)

    # Save the resulting image
    result_filename = f"result_{source_image_file}_{target_image_file}"
    output_path = os.path.join(output_folder, result_filename)
    cv2.imwrite(output_path, result)

print("Image processing completed!")


Processing source image: 1 (1006).jpg
Skipping target image: 1 (1265).jpg
Processing source image: 1 (1007).jpg
Processing target image: 1 (1266).jpg


100%|██████████| 1/1 [00:00<00:00, 50.51it/s]
1it [00:00, 146.74it/s]
1it [00:00, 367.18it/s]
100%|██████████| 1/1 [00:00<00:00, 1483.66it/s]
1it [00:00,  6.14it/s]


Processing source image: 1 (1008).jpg
Processing target image: 1 (1267).jpg


100%|██████████| 1/1 [00:00<00:00, 43.97it/s]
1it [00:00, 159.76it/s]
1it [00:00, 322.02it/s]
100%|██████████| 1/1 [00:00<00:00, 1726.76it/s]
1it [00:00,  6.09it/s]


Processing source image: 1 (1009).jpg
Processing target image: 1 (1268).jpg


...

Processing source image: 1 (1262).jpg
Processing target image: 1 (1516).jpg


100%|██████████| 1/1 [00:00<00:00, 45.56it/s]
1it [00:00, 189.21it/s]
1it [00:00, 2194.82it/s]
100%|██████████| 1/1 [00:00<00:00, 9986.44it/s]
1it [00:00,  5.71it/s]


Processing source image: 1 (1263).jpg
Skipping target image: 1 (1517).jpg
Processing source image: 1 (1264).jpg
Processing target image: 1 (1518).jpg


100%|██████████| 1/1 [00:00<00:00, 35.51it/s]
1it [00:00, 226.62it/s]
1it [00:00, 988.76it/s]
100%|██████████| 1/1 [00:00<00:00, 2484.78it/s]
1it [00:00,  4.45it/s]


Image processing completed!


# Real vs Fake model training

In [ ]:
%cd /content

/content


In [ ]:
!pwd

/content


In [ ]:
!mkdir Real_vs_fake



In [ ]:
%cd /content/Real_vs_fake

/content/Real_vs_fake


In [ ]:
!gdown --id 1y6IUt3WOF5LUjnLqlBNxjJrx7OObc5gq


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1y6IUt3WOF5LUjnLqlBNxjJrx7OObc5gq
To: /content/deepfake.zip
100% 1.69G/1.69G [00:17<00:00, 99.4MB/s]


In [ ]:
!unzip /content/Real_vs_fake/deepfake.zip

unzip:  cannot find or open /content/Real_vs_fake/deepfake.zip, /content/Real_vs_fake/deepfake.zip.zip or /content/Real_vs_fake/deepfake.zip.ZIP.


In [ ]:
!pip install ultralytics==8.0.114

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.4/595.4 kB 33.9 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
!yolo classify train data="/content/Real_vs_fake/Dataset" model=yolov8m-cls.pt epochs=50  batch=8 lr0=0.001  lrf=0.001 cos_lr=True imgsz=64


Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/cfg/__init__.py", line 342, in entrypoint
    check_cfg_mismatch(full_args_dict, overrides)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/cfg/__init__.py", line 185, in check_cfg_mismatch
    raise SyntaxError(string + CLI_HELP_MSG) from e
SyntaxError: 'img' is not a valid YOLO argument. Similar arguments are i.e. ['imgsz=640'].

    Arguments received: ['yolo', 'classify', 'train', 'data=/content/Real_vs_fake/Dataset', 'model=yolov8m-cls.pt', 'epochs=50', 'batch=8', 'lr0=0.001', 'lrf=0.001', 'cos_lr=True', 'img=64']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
          

# Model testing

In [ ]:
def Predict(img):
    model = YOLO("/content/drive/MyDrive/FYP/runs/classify/train2/weights/best.pt")
    results = model(img, verbose=False)
    #print(results)
    res_plotted = results[0].plot()

    #result_dict = {}
    for result in results:
        class_name = result.names[result.probs.top1]
        #print(classes_names)
        probability = result.probs.top1conf.item()
        #print(probs)
        #result_dict[classes_names] = probs


    img = mpimg.imread(img)

    # Plot the image
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()
    return {"Class": class_name, "Confidence": probability}